In [4]:
import pandas as pd, matplotlib.pyplot as plt, numpy as numpy

df = pd.read_csv("data/library_loans.csv", parse_dates=["checkout_date", "returned_date"])

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   loan_id           129 non-null    object        
 1   checkout_date     129 non-null    datetime64[ns]
 2   branch            126 non-null    object        
 3   genre             123 non-null    object        
 4   item_type         128 non-null    object        
 5   patron_age_group  128 non-null    object        
 6   loan_days         129 non-null    int64         
 7   returned_date     113 non-null    datetime64[ns]
 8   overdue_days      119 non-null    float64       
 9   fine_amount       117 non-null    float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(5)
memory usage: 10.2+ KB
